In [5]:
import pandas as pd
import os
#import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter


from sqlalchemy import create_engine, text

#maximum number of rows to display
pd.options.display.max_rows = 50

DB_USERNAME = 'developers'
DB_PASSWORD = '12345678'
DB_INSTANCE = 'database-1.cmostnjfvkd6.us-east-2.rds.amazonaws.com'

#DB_INSTANCE = 'localhost'
DB_PORT = '5432'
DB_DATA_BASE = 'cnc'

engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_INSTANCE}:{DB_PORT}/{DB_DATA_BASE}')

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())


query_03= """
SELECT * FROM efectivas_final;
"""
df_efectivas = runQuery(query_03)

In [7]:
df_efectivas['mes']= pd.DatetimeIndex(df_efectivas['fecha']).month
df_efectivas['dia']= pd.DatetimeIndex(df_efectivas['fecha']).day
df_efectivas['año']= pd.DatetimeIndex(df_efectivas['fecha']).year
df_efectivas_groupm=df_efectivas.groupby(['codenc','mes','año']).agg({'Edad':'count'}).reset_index().rename(columns={'Edad':'count'})

df_efectivas_groupm[(df_efectivas_groupm['año']==2019) & (df_efectivas_groupm['mes'].isin([1,2,3,4,5,6,7,8,9,10,11,12]))].sort_values('count',ascending=False)


,codenc,mes,año,count
833,Encuestador 270,4,2019,268
835,Encuestador 270,6,2019,241
839,Encuestador 270,10,2019,240
834,Encuestador 270,5,2019,238
302,Encuestador 169,4,2019,228
...,...,...,...,...
653,Encuestador 238,1,2019,1
1194,Encuestador 47,1,2019,1
494,Encuestador 201,11,2019,1
593,Encuestador 224,1,2019,1


In [8]:
df_efectivas['Edad'].replace('12 a 17 años','12 a 17',inplace=True)
df_efectivas

,codenc,telelink,fecha,duracion,Ciudad,Estrato,Genero,Edad,mes,dia,año
0,Encuestador 35,227228.0,2018-07-10,558,Cali,Bajo,Mujer,12 a 17,7,10,2018
1,Encuestador 83,248904.0,2018-10-12,595,Cali,Medio Alto,Mujer,12 a 17,10,12,2018
2,Encuestador 30,257365.0,2018-11-19,433,Neiva,Medio Bajo,Mujer,12 a 17,11,19,2018
3,Encuestador 236,155288.0,2017-08-29,732,Pasto,Medio Bajo,Hombre,35 a 44,8,29,2017
4,Encuestador 84,194815.0,2018-02-18,591,Bucaramanga,Bajo,Hombre,25 a 34,2,18,2018
...,...,...,...,...,...,...,...,...,...,...,...
142256,Encuestador 197,233552.0,2018-08-08,993,Tunja,Bajo,Mujer,25 a 34,8,8,2018
142257,Encuestador 159,177987.0,2017-12-07,611,Tunja,Bajo,Hombre,25 a 34,12,7,2017
142258,Encuestador 260,305450.0,2019-06-21,940,Popayán,Bajo,Hombre,12 a 17,6,21,2019
142259,Encuestador 288,330095.0,2019-09-14,1022,Popayán,Bajo,Hombre,45 a 54,9,14,2019


In [9]:
query_04= """select *
from ecar_dwh.ecar_dwh_base_encuestas_efectivas """
df_efectivas_pollster = runQuery(query_04)
df_efectivas_pollster

,codenc,telelink,fecha,duracion,genero,edad,ciudad,estrato,cod_rango_edad,archivo
0,Encuestador 280,163503.0,2017-10-06,618,1,15,1,3,1,Productividad_Octubre_2017.xlsx
1,Encuestador 84,163505.0,2017-10-06,901,1,25,1,2,3,Productividad_Octubre_2017.xlsx
2,Encuestador 223,163508.0,2017-10-06,478,1,12,1,2,1,Productividad_Octubre_2017.xlsx
3,Encuestador 282,163510.0,2017-10-06,412,2,35,1,1,4,Productividad_Octubre_2017.xlsx
4,Encuestador 116,163512.0,2017-10-06,812,1,28,1,2,3,Productividad_Octubre_2017.xlsx
...,...,...,...,...,...,...,...,...,...,...
142256,Encuestador 135,155628.0,2017-08-31,608,1,38,1,5,4,Productividad_Agosto_2017.xlsx
142257,Encuestador 95,155629.0,2017-08-31,1011,1,35,1,5,4,Productividad_Agosto_2017.xlsx
142258,Encuestador 100,155630.0,2017-08-31,453,1,16,1,3,1,Productividad_Agosto_2017.xlsx
142259,Encuestador 292,155631.0,2017-08-31,894,2,37,1,6,4,Productividad_Agosto_2017.xlsx


In [12]:
df_efectivas_cuotas=pd.DataFrame()
for i in ['Estrato', 'Genero','Edad']:
    df_count=df_efectivas.groupby(i)[['codenc']].count().rename(columns={'codenc':"count"})
    df_count['variable']=i
    df_efectivas_cuotas=pd.concat([df_count, df_efectivas_cuotas])

In [13]:
df_efectivas_cuotas=df_efectivas_cuotas.sort_values('count',ascending=True)

In [14]:
df_efectivas[(df_efectivas['codenc']=='Encuestador 137') | (df_efectivas['codenc']=='Encuestador 10')]

,codenc,telelink,fecha,duracion,Ciudad,Estrato,Genero,Edad,mes,dia,año
9820,Encuestador 10,NaN,2018-05-06,961,Medellín,Alto,Hombre,45 a 54,5,6,2018
11080,Encuestador 10,211098.0,2018-04-30,916,Medellín,Bajo,Mujer,18 a 24,4,30,2018
15490,Encuestador 10,NaN,2018-05-06,648,Bogotá,Medio Alto,Mujer,45 a 54,5,6,2018
18908,Encuestador 137,NaN,2018-12-18,945,Bogotá,Bajo,Mujer,12 a 17,12,18,2018
25056,Encuestador 10,210910.0,2018-04-29,1025,Medellín,Bajo,Hombre,45 a 54,4,29,2018
25433,Encuestador 10,210942.0,2018-04-29,1109,Medellín,Medio Bajo,Hombre,45 a 54,4,29,2018
26929,Encuestador 137,NaN,2018-12-16,1045,Medellín,Bajo,Mujer,25 a 34,12,16,2018
39086,Encuestador 137,NaN,2018-12-19,1878,Barranquilla,Bajo,Mujer,12 a 17,12,19,2018
44588,Encuestador 137,NaN,2018-12-18,815,Bogotá,Bajo,Hombre,12 a 17,12,18,2018
49803,Encuestador 137,NaN,2018-12-12,1307,Bogotá,Medio Medio,Mujer,25 a 34,12,12,2018


In [15]:

query_02=""" select * from ecar_dwh.ecar_dwh_base_modelo """
df_modelo=runQuery(query_02)

cols=['calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas','efectividad_ajustada']

df_tiempos=df_modelo[cols]

In [16]:
df_tiempos

,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,efectividad_ajustada
0,6252.705882,623484.000000,150572.000000,313690.000000,42598.000000,116624.000000,2193.000000,1102.323944,1804.235294
1,10989.000000,147804.000000,40859.500000,61113.500000,14951.500000,30879.500000,289.000000,954.035714,1188.000000
2,6900.927106,820764.181818,235728.090909,330865.454545,88244.272727,165926.363636,7262.818182,739.523805,2980.843057
3,4202.400000,125574.500000,37047.000000,49981.500000,9136.000000,29410.000000,3586.500000,836.228571,2268.000000
4,6440.057143,510847.500000,180869.500000,202350.000000,30753.500000,96874.500000,5939.500000,749.815857,1627.714286
...,...,...,...,...,...,...,...,...,...
279,7177.520000,760716.000000,226663.000000,311614.333333,82926.000000,139512.666667,4070.333333,719.768171,3102.240000
280,6085.707762,501703.800000,156219.600000,174043.800000,79735.400000,91705.000000,5142.000000,818.476296,2383.644187
281,9660.391111,554952.000000,112426.666667,252206.666667,69592.000000,120726.666667,4709.000000,1077.662688,2026.880000
282,4788.000000,604918.000000,134926.000000,305491.000000,40198.000000,124303.000000,2648.000000,993.446809,2256.000000


In [17]:
df_tiempos[df_tiempos['efectividad_ajustada'].isnull()==True]

,calls,total,espera,hablado,disponible,pausas,muerto,duracion_efectivas,efectividad_ajustada
198,NaN,32001.0,11897.0,9051.0,4327.0,6726.0,8.0,NaN,NaN


In [18]:
cols=['efectivas_mujer', 'efectivas_hombre','efectivas_alto', 'efectivas_medio_alto', 'efectivas_medio_medio','efectivas_bajo', 'efectivas_medio_bajo', 'efectivas_tunja',
       'efectivas_popayán', 'efectivas_montería', 'efectivas_manizales','efectivas_armenia', 'efectivas_villavicencio', 'efectivas_pasto','efectivas_cúcuta', 'efectivas_santa_marta', 'efectivas_ibagué',
       'efectivas_neiva', 'efectivas_pereira', 'efectivas_cartagena','efectivas_cali', 'efectivas_bucaramanga', 'efectivas_barranquilla','efectivas_medellín', 'efectivas_bogotá', 'efectivas_rango_edad_5',
       'efectivas_rango_edad_2', 'efectivas_rango_edad_4','efectivas_rango_edad_3', 'efectivas_rango_edad_1']
df_modelo=df_modelo[df_modelo['codenc']!='Encuestador 290']
for i in cols:
    df_modelo["%" + str(i)]=round((df_modelo[i]/df_modelo['efectividad_ajustada'])*100).astype(int)
df_porc_tabla=df_modelo[['codenc','efectividad_ajustada','meses_trabajados','como_conocio_cnc','localidad','educacion_formal','estado_educacion','nombre_educacion',
         '%efectivas_mujer', '%efectivas_hombre', '%efectivas_alto','%efectivas_medio_alto', '%efectivas_medio_medio',
         '%efectivas_bajo', '%efectivas_medio_bajo','%efectivas_tunja', '%efectivas_popayán','%efectivas_montería', 
         '%efectivas_manizales','%efectivas_armenia', '%efectivas_villavicencio','%efectivas_pasto', '%efectivas_cúcuta',
         '%efectivas_santa_marta', '%efectivas_ibagué','%efectivas_neiva', '%efectivas_pereira','%efectivas_cartagena', '%efectivas_cali',
         '%efectivas_bucaramanga', '%efectivas_barranquilla','%efectivas_medellín', '%efectivas_bogotá','%efectivas_rango_edad_5', '%efectivas_rango_edad_2',
         '%efectivas_rango_edad_4', '%efectivas_rango_edad_3','%efectivas_rango_edad_1']]

In [56]:
pd.options.display.max_rows =20
df_modelo.columns

Index(['codenc', 'calls', 'total', 'espera', 'hablado', 'disponible', 'pausas',
       'muerto', 'duracion_efectivas', 'dias_trabajados',
       'efectividad_ajustada', 'efectivas_mujer', 'efectivas_hombre',
       'efectivas_alto', 'efectivas_medio_alto', 'efectivas_medio_medio',
       'efectivas_bajo', 'efectivas_medio_bajo', 'efectivas_tunja',
       'efectivas_popayán', 'efectivas_montería', 'efectivas_manizales',
       'efectivas_armenia', 'efectivas_villavicencio', 'efectivas_pasto',
       'efectivas_cúcuta', 'efectivas_santa_marta', 'efectivas_ibagué',
       'efectivas_neiva', 'efectivas_pereira', 'efectivas_cartagena',
       'efectivas_cali', 'efectivas_bucaramanga', 'efectivas_barranquilla',
       'efectivas_medellín', 'efectivas_bogotá', 'efectivas_rango_edad_5',
       'efectivas_rango_edad_2', 'efectivas_rango_edad_4',
       'efectivas_rango_edad_3', 'efectivas_rango_edad_1', 'meses_trabajados',
       'competencias_funcionales', 'numeros', 'lectura_voz_alta',
      

In [64]:
df_encuestadores_efec=df_porc_tabla[['codenc','efectividad_ajustada','%efectivas_hombre','%efectivas_mujer']].sort_values('efectividad_ajustada',ascending=False)
df_encuestadores_efec['suma']=df_encuestadores_efec['%efectivas_hombre']+df_encuestadores_efec['%efectivas_mujer']
df_encuestadores_efec[df_encuestadores_efec['suma']!=100]

,codenc,efectividad_ajustada,%efectivas_hombre,%efectivas_mujer,suma
177,Encuestador 270,3703.006175,41,52,93
189,Encuestador 282,3355.934452,47,44,91
72,Encuestador 169,3291.822163,48,46,94
108,Encuestador 204,3263.709849,43,38,81
145,Encuestador 240,3182.787692,37,46,83
...,...,...,...,...,...
1,Encuestador 10,1188.000000,18,18,36
40,Encuestador 137,987.428571,0,0,0
33,Encuestador 130,911.153110,35,44,79
220,Encuestador 311,864.000000,0,0,0
